In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras import layers, models
import requests
from io import BytesIO
from zipfile import ZipFile

In [6]:
# Загрузка и подготовка данных
from google.colab import drive
drive.mount('/content/drive')

data_path = "/content/drive/My Drive/Project/DataSetCyrillic"

Mounted at /content/drive


In [54]:
# размер изображений 28 на 28 => входной вектор длины 784
labels = []
images = []

for folder in os.listdir(data_path):
    for filename in os.listdir(os.path.join(data_path, folder)):
        img_path = os.path.join(data_path, folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img / 255  # Нормализация значений пикселей
        label = ord(folder[0]) - ord('А')
        images.append(img)
        labels.append(label)

In [55]:
# Преобразование в массив numpy
images = np.array(images)
labels = np.array(labels)

In [56]:
# Разделение данных на обучающий и тестовый наборы
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

In [50]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten

In [51]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))  # скрытый слой 128 нейронов
model.add(layers.Dense(66, activation='softmax'))  # выходной 63 нейрона (33 буквы, из них все, кроме ъ, ы, ь в двух вариантах (большая и маленькая))
# 66 для краткости

print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 5408)              0         
                                                                 
 dense_16 (Dense)            (None, 128)               692352    
                                                                 
 dense_17 (Dense)            (None, 66)                8514      
                                                                 
Total params: 701186 (2.67 MB)
Trainable params: 701186 (2.67 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [57]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [58]:
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split=0.2)

Epoch 1/5
493/493 [==============================] - 12s 23ms/step - loss: 3.9318 - accuracy: 0.0535 - val_loss: 3.6457 - val_accuracy: 0.0959
Epoch 2/5
493/493 [==============================] - 12s 25ms/step - loss: 3.3934 - accuracy: 0.1405 - val_loss: 3.2220 - val_accuracy: 0.1743
Epoch 3/5
493/493 [==============================] - 14s 28ms/step - loss: 3.0073 - accuracy: 0.2227 - val_loss: 2.8803 - val_accuracy: 0.2547
Epoch 4/5
493/493 [==============================] - 12s 24ms/step - loss: 2.7310 - accuracy: 0.2917 - val_loss: 2.6913 - val_accuracy: 0.2950
Epoch 5/5
493/493 [==============================] - 12s 23ms/step - loss: 2.5431 - accuracy: 0.3337 - val_loss: 2.5565 - val_accuracy: 0.3283
